In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from tensorflow.python.client import device_lib
print device_lib.list_local_devices()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2735945177728515778
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11983277261
locality {
  bus_id: 1
}
incarnation: 8933009795661634402
physical_device_desc: "device: 0, name: TITAN X (Pascal), pci bus id: 0000:05:00.0, compute capability: 6.1"
]


In [2]:
import tensorflow as tf
import numpy as np
from pdrnn import triangular_pmf, build_pdrnn, compute_loss

In [4]:
# build the network
tf.reset_default_graph()
inputs = tf.placeholder(tf.float32, shape = [32, 20, 1], name = 'inputs')

logits, structs, rates = build_pdrnn(x=inputs,
                                     hidden_structs=[4]*2,
                                     init_params=[(2.0,2.0),(2.0,2.0)],
                                     n_layers=2,
                                     n_steps=20,
                                     n_classes=2,
                                     input_dims=1,
                                     cell_type="RNN")

(32, ?, 1)
(?, ?, 1)
(?, ?, 1)
(?, 1)
(?, 4)


TypeError: Expected int for argument 'num_split' not <tf.Tensor 'strided_slice_2:0' shape=() dtype=int32>.

In [40]:
a = tf.ones([10, 16])
b = tf.split(a, [1]*10)

print(b)

[<tf.Tensor 'split_1:0' shape=(1, 16) dtype=float32>, <tf.Tensor 'split_1:1' shape=(1, 16) dtype=float32>, <tf.Tensor 'split_1:2' shape=(1, 16) dtype=float32>, <tf.Tensor 'split_1:3' shape=(1, 16) dtype=float32>, <tf.Tensor 'split_1:4' shape=(1, 16) dtype=float32>, <tf.Tensor 'split_1:5' shape=(1, 16) dtype=float32>, <tf.Tensor 'split_1:6' shape=(1, 16) dtype=float32>, <tf.Tensor 'split_1:7' shape=(1, 16) dtype=float32>, <tf.Tensor 'split_1:8' shape=(1, 16) dtype=float32>, <tf.Tensor 'split_1:9' shape=(1, 16) dtype=float32>]


In [23]:
t = tf.constant([[1, 2, 3], [4, 5, 6]])
paddings = tf.constant([[0, 0], [0, 3]])
with tf.Session():
    print tf.pad(t, paddings, constant_values=0.0).eval() 

[[1 2 3 0 0 0]
 [4 5 6 0 0 0]]


In [113]:
tf.reset_default_graph()
init_params=[(2.0,2.0),(2.0,2.0)]
dilations = []
for l in xrange(2):
    with tf.variable_scope('struct_layer_{}'.format(l)):
        mu = tf.get_variable('mu', initializer=tf.constant(init_params[l][0]))
        sigma = tf.get_variable('sigma', initializer=tf.constant(init_params[l][1]))
                
    rates = tf.range(tf.ceil(mu-sigma), tf.ceil(mu+sigma))
    probs = triangular_pmf(rates, mu, sigma)
    probs = tf.expand_dims(probs, 0)
    idx = tf.multinomial(tf.log(probs), 1)
    rates = tf.cast(rates, tf.int32)
    dilations.append(rates[idx[0][0]])  

In [126]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print rates.eval()
    print probs.eval()
    print sess.run(dilations)

[0 1 2 3]
[[0.   0.25 0.5  0.25]]
[1, 2]


In [9]:
from classification_models import drnn_classification
# build the network
tf.reset_default_graph()

dilations = []
for l in xrange(2):
    d = tf.placeholder_with_default(tf.constant(2), [])
    dilations.append(d)


inputs = tf.placeholder(tf.float32, shape = [32, 20, 1], name = 'inputs')

logits = drnn_classification(x=inputs,
                             hidden_structs=[4]*2,
                             dilations=dilations,
                             n_steps=20,
                             n_classes=2,
                             input_dims=1,
                             cell_type='RNN')

(32, ?, 1)
(?, ?, 1)
(?, ?, 1)
(?, 1)
(?, 4)


ValueError: None values not supported.